In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(0,'../../')

In [2]:
import pandas as pd

In [3]:
from medline.dataprep import TRAIN_TEXTS_FILE
from gensim.models.word2vec import LineSentence

In [4]:
docs = LineSentence(TRAIN_TEXTS_FILE)

In [5]:
from gensim.models.word2vec import Word2Vec

In [6]:
model = Word2Vec(min_count=0, sample=0)
model.scan_vocab(docs)

In [7]:
unique_words = len(model.raw_vocab)
min_count = list(range(1,20))

rows = []
for m in min_count:
    report = model.scale_vocab(min_count=m, dry_run=True)

    drop_unique = report['drop_unique']
    retain_total = report['retain_total']
    retain_unique = unique_words - drop_unique
    row = [drop_unique, retain_unique, retain_total]
    rows.append(row)
    
columns = ['drop_unique', 'retain_unique', 'retain_total']
df = pd.DataFrame(rows, min_count, columns)

In [8]:
df

,drop_unique,retain_unique,retain_total
1,0,2703192,2165549630
2,1135235,1567957,2164414395
3,1521762,1181430,2163641341
4,1733123,970069,2163007258
5,1872763,830429,2162448698
6,1972352,730840,2161950753
7,2047580,655612,2161499385
8,2105003,598189,2161097424
9,2151226,551966,2160727640
10,2188526,514666,2160391940


In [9]:
sampling=[1e-3, 1e-4, 1e-5]

rows = []
for s in sampling:
    report = model.scale_vocab(sample=s, dry_run=True)

    downsample_unique = report['downsample_unique']
    downsample_total = report['downsample_total']
    row = [downsample_unique, downsample_total]
    rows.append(row)

columns = ['downsample_unique', 'downsample_total']
df = pd.DataFrame(rows, sampling, columns)

In [10]:
df

,downsample_unique,downsample_total
0.00100,30,1605217174
0.00010,414,1197058791
0.00001,3635,668181115


In [11]:
model.scale_vocab(min_count=14)

{'downsample_total': 2159268084,
 'downsample_unique': 0,
 'drop_unique': 2287939,
 'memory': {'syn0': 166101200,
  'syn1neg': 166101200,
  'total': 539828900,
  'vocab': 207626500},
 'retain_total': 2159268084}

In [12]:
word_count = {k: v.count for k, v in model.wv.vocab.items()}
word_count = sorted(word_count.items(), key=lambda d: -d[1])
word, count = zip(*word_count)

df = pd.DataFrame(list(count), word, ['word'])

In [13]:
df.head()

,word
.,108329162
the,97640187
of,84912442
",",78334878
and,63622627


In [14]:
df.tail()

,word
browner,14
ertr,14
wealthmod,14
hpv20,14
tmia,14


In [15]:
import numpy as np

lengths = [len([w for w in doc if w in model.wv.vocab])
           for doc in docs]

percentiles = [50, 95, 99, 99.5, 99.9, 99.99]

rows = []
for percentile in percentiles:
    rows.append(int(np.percentile(lengths, percentile)))
    
df = pd.DataFrame(rows, percentiles, ['length'])

In [16]:
df

,length
50.00,229
95.00,395
99.00,498
99.50,547
99.90,691
99.99,1034
